AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Import artificially generated FHIR Resources - 1000 patients generated with Synthea

In [1]:
!python --version

Python 3.10.6


In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': '41cf76cb1205',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.3.0',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'PATHLING_VERSION': '5.4.0',
        'SPARK_CONF_DIR': '/usr/local/spark/conf',
        'NB_USER': 'jovyan',
        'LC_ALL': 'en_US.UTF-8',
        'PATH': '/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/spark/bin',
        'PYSPARK_PYTHONPATH_SET': 

In [4]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
#kafka_topic = "fhir.test.pathling" #"fhir.post-gateway-specimen"
#kafka_server = "kafka1:19092"
#kafka_server = "host.docker.internal:61687" # port that kafka is forwarded to in Lens

In [5]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pathling.etc import find_jar

#conf = SparkConf()
#conf.setMaster(master) \
#    .setAppName(appName) \
#    .set("spark.ui.port","0")

# get current jars
#spark_jars = conf.get("spark.jars")
# append pathling jar
#updated_spark_jars = spark_jars + ",file://" + find_jar()

#updated_spark_jars = ["/test/1.jar", "/test2.jar"]

spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.ui.port","0") \
    .getOrCreate()

spark.sparkContext.addFile(find_jar())

# Bring Pathling into the game

In [6]:
import pathling
from pathling import PathlingContext
from pyspark.sql.functions import regexp_replace, col, explode, concat_ws

ptl = PathlingContext.create(spark = spark)

### Pathling Encoder: Convert json Bundles from Synthea into structured FHIR resources
### FIRST TEST: with 1 single bundle (1 patient + corresponding other resource types)

In [7]:
!pwd

/home/jovyan/work


In [10]:
!ls synthea/

fhir  metadata	one_json_test


In [11]:
bundles_dir = 'synthea/fhir/'
bundles = ptl.spark.read.text(bundles_dir, wholetext=True)

### PATIENTS RESOURCES

In [12]:
patients = ptl.encode_bundle(bundles, 'Patient')

In [13]:
print(type(patients))

<class 'pyspark.sql.dataframe.DataFrame'>


In [26]:
#patients.printSchema()

In [27]:
patients_selection = patients.select(
                                    col("id").alias("patient_id"), \
                                    'gender', 'birthDate', 'deceasedDateTime')

In [28]:
print(patients_selection.columns)

['patient_id', 'gender', 'birthDate', 'deceasedDateTime']


In [29]:
patients_selection.show()

+--------------------+------+----------+--------------------+
|          patient_id|gender| birthDate|    deceasedDateTime|
+--------------------+------+----------+--------------------+
|urn:uuid:3bc94ae3...|  male|1949-05-05|                null|
|urn:uuid:249c1400...|  male|1922-11-18|                null|
|urn:uuid:fc5937fc...|female|1953-03-27|                null|
|urn:uuid:8588dc78...|female|1950-06-12|2012-10-14T02:21:...|
|urn:uuid:dfcde42f...|  male|1927-04-11|1993-09-12T08:04:...|
|urn:uuid:0362e73e...|female|1913-09-13|                null|
|urn:uuid:6dfd981a...|  male|1948-08-07|                null|
|urn:uuid:314cff1c...|female|1942-01-13|                null|
|urn:uuid:bdae5890...|female|1956-02-28|                null|
|urn:uuid:a2a11a91...|female|1953-04-06|                null|
|urn:uuid:a2b725c5...|  male|1956-07-29|                null|
|urn:uuid:91961269...|  male|1940-12-18|2018-12-10T20:46:...|
|urn:uuid:b360962d...|female|1953-03-27|2014-05-15T13:43:...|
|urn:uui

In [22]:
# läd das hier schon alle in memory?? dauert es deshalb lang oder macht das hier noch gar nichts
#patients_selection.toPandas()

### ENCOUNTER - brauch ich vll nicht?

In [30]:
encounters = ptl.encode_bundle(bundles, 'Encounter')

In [ ]:
#encounters.printSchema()

In [31]:
encounters_selection = encounters.select(
                                        col("id").alias("enc_id"), \
                                        "subject.reference", "period.start", "period.end")

In [32]:
encounters_selection.toPandas()

,enc_id,reference,start,end
0,urn:uuid:4d2c5eb2-295b-5c6f-e9e2-f39da3f3aeaa,urn:uuid:3bc94ae3-0aa8-095b-a6f4-adc9a9468991,1949-06-07T19:07:56+02:00,1949-06-07T19:22:56+02:00
1,urn:uuid:c24faf25-239d-c036-8403-0c2553382b49,urn:uuid:3bc94ae3-0aa8-095b-a6f4-adc9a9468991,1961-02-15T22:35:56+01:00,1961-02-15T23:35:56+01:00
2,urn:uuid:94ebbaed-b5b4-811b-1224-e886db7ffd6b,urn:uuid:3bc94ae3-0aa8-095b-a6f4-adc9a9468991,1962-10-10T01:07:56+01:00,1962-10-10T01:22:56+01:00
3,urn:uuid:7f72fba6-fb43-0ee9-feef-208e5f26f32c,urn:uuid:3bc94ae3-0aa8-095b-a6f4-adc9a9468991,1964-06-11T20:07:56+01:00,1970-01-01T01:00:00+01:00
4,urn:uuid:cfb2c8ef-e4f4-2107-4582-ee445f5bb6e1,urn:uuid:3bc94ae3-0aa8-095b-a6f4-adc9a9468991,1964-09-30T20:07:56+01:00,1964-09-30T20:22:56+01:00
...,...,...,...,...
61216,urn:uuid:d7b8d34f-b5bb-0ba8-3777-1031c23a3863,urn:uuid:847a8fd2-d5b9-f6f3-06e2-59723f2730df,2023-01-07T05:38:17+01:00,2023-01-07T05:53:17+01:00
61217,urn:uuid:c5ac4c9f-b2e7-7b2d-5840-1ca9c50d1135,urn:uuid:533ef656-1fbc-7c99-dbcb-c6fa35dc13cb,2022-10-20T14:04:41+02:00,2022-10-20T14:19:41+02:00
61218,urn:uuid:3085c97e-b13e-197b-97e2-58bdc6678c6a,urn:uuid:533ef656-1fbc-7c99-dbcb-c6fa35dc13cb,2022-11-24T13:04:41+01:00,2022-11-24T13:19:41+01:00
61219,urn:uuid:4c9e13a9-7678-5835-4c70-524fecbb2ef8,urn:uuid:b311e788-2736-0ca2-a739-b6bdefe21f17,2022-11-25T23:10:59+01:00,2022-11-25T23:25:59+01:00


### CONDITIONS

In [33]:
conditions = ptl.encode_bundle(bundles, 'Condition')

In [48]:
#conditions.printSchema()

In [51]:
conditions_selection = conditions.select(
                                        col("id").alias("condition_id"), \
                                        col("subject.reference").alias("condition_pat_id"), \
                                        col("code.coding.system").alias("condition_code_system"), \
                                        col("code.coding.code").alias("condition_code"))

In [52]:
conditions_selection.show()

+--------------------+--------------------+---------------------+-----------------+
|        condition_id|    condition_pat_id|condition_code_system|   condition_code|
+--------------------+--------------------+---------------------+-----------------+
|urn:uuid:9ea309b1...|urn:uuid:3bc94ae3...| [http://snomed.in...|      [160968000]|
|urn:uuid:804e6bce...|urn:uuid:3bc94ae3...| [http://snomed.in...|      [160968000]|
|urn:uuid:0880b230...|urn:uuid:3bc94ae3...| [http://snomed.in...|       [15777000]|
|urn:uuid:ba14b272...|urn:uuid:3bc94ae3...| [http://snomed.in...|      [271737000]|
|urn:uuid:884d1c22...|urn:uuid:3bc94ae3...| [http://snomed.in...|  [5251000175109]|
|urn:uuid:c62336c1...|urn:uuid:3bc94ae3...| [http://snomed.in...|      [160904001]|
|urn:uuid:4c69cbc2...|urn:uuid:3bc94ae3...| [http://snomed.in...|      [266948004]|
|urn:uuid:2e5b3e43...|urn:uuid:3bc94ae3...| [http://snomed.in...|       [44054006]|
|urn:uuid:5c1bd6cb...|urn:uuid:3bc94ae3...| [http://snomed.in...|       [596

### AllergyIntolerance

In [57]:
allergyIntolerances = ptl.encode_bundle(bundles, 'AllergyIntolerance')

In [58]:
#allergyIntolerances.printSchema()

In [65]:
allergyIntolerances_selection = allergyIntolerances.select("type", "category", "criticality", "code.coding.system", "code.coding.code", "patient.reference", "reaction")

In [66]:
allergyIntolerances_selection.show()

+-----------+-------------+-----------+--------------------+-----------+--------------------+--------------------+
|       type|     category|criticality|              system|       code|           reference|            reaction|
+-----------+-------------+-----------+--------------------+-----------+--------------------+--------------------+
|    allergy|[environment]|        low|[http://snomed.in...| [84489001]|urn:uuid:0362e73e...|                  []|
|    allergy|       [food]|        low|[http://snomed.in...|[735029006]|urn:uuid:0362e73e...|[{null, {null, []...|
|    allergy|[environment]|        low|[http://snomed.in...|[256277009]|urn:uuid:bdae5890...|                  []|
|    allergy|       [food]|        low|[http://snomed.in...|[735029006]|urn:uuid:bdae5890...|[{null, {null, []...|
|    allergy|       [food]|        low|[http://snomed.in...|[762952008]|urn:uuid:94318223...|[{null, {null, []...|
|    allergy|[environment]|        low|[http://snomed.in...| [84489001]|urn:uuid